# Simpleksna metoda

Sledeči program po korakih izvaja simpleksno metodo.

In [ ]:
class DrugaFaza:
    """
    Razred, ki označuje prehod simpleksne metode v drugo fazo.
    """
    def __init__(self, x0=var("x0"), w=var("w")):
        """
        Konstruktor za specifikacijo pomožne spremenljivke in pomožnega funkcionala.
        """
        self.x0 = x0
        self.w = w

def spremenljivke(n, zacetek=0, predpona="x", ciljna="z"):
    """
    Nastavi želeno število spremenljivk s podano predpono.
    """
    var(" ".join("%s%d" % (predpona, i) for i in range(zacetek, n+1)))
    var(ciljna)

def izpisi_enacbe(enacbe):
    """
    Uredi in ustrezno izpiše enačbe.
    """
    enacbe.sort(key=lambda e: str(e.lhs()))
    for e in enacbe:
        r = e.rhs()
        c = r.subs([v == 0 for v in r.variables()])
        rr = r - c
        s = str(rr)
        if c == 0:
            c = znak = ""
        elif s[0] == "-":
            s = s[1:]
            znak = " - "
        else:
            znak = " + "
        print("%s == %s%s%s" % (e.lhs(), c, znak, s))
    print("")
    return enacbe

def simpleksna_metoda(enacbe, *spremenljivke):
    """
    Za podani začetni slovar izvaja podane zamenjave baznih in nebaznih spremenljivk.
    """
    enacbe = izpisi_enacbe(enacbe)
    for spr in spremenljivke:
        if isinstance(spr, DrugaFaza):
            enacbe = izpisi_enacbe([e.subs(spr.x0 == 0) for e in enacbe if e.lhs() != spr.w])
        else:
            vstopna, izstopna = spr
            sol = next(e for e in enacbe if e.lhs() == izstopna).solve(vstopna)[0]
            enacbe = izpisi_enacbe([sol if e.lhs() == izstopna else e.subs(sol) for e in enacbe])

Funkcija `izpisi_enacbe` je pomožna. S funkcijo `spremenljivke` si pripravimo želeno število spremenljivk, funkcijo `simpleksna_metoda` pa uporabimo tako, da kot prvi argument podamo slovar enačb, za tem pa naštevamo pare vstopnih in izstopnih spremenljivk. Poleg tega lahko podamo tudi instanco razreda `DrugaFaza`, ki iz slovarja odstrani pomožno spremenljivko (privzeto `x0`) in pomožni funkcional (privzeto `w`).

## Reševanje linearnega programa

Poiščimo rešitev spodnjega linearnega programa.
$$
\begin{aligned}
\max &\ & 10 x_1 + 15 x_2 + 12 x_3 \\[1ex]
\text{p.p.} && x_1 + x_2 + x_3 &\le 50 \\
&& 3 x_1 + 4 x_2 + 5 x_3 &\le 250 \\
&& 3 x_1 + 5 x_2 + 4 x_3 &\le 300 \\
&& x_1, x_2, x_3 &\ge 0
\end{aligned}
$$

In [13]:
spremenljivke(6)
simpleksna_metoda([
    x4 == 50 - x1 - x2 - x3,
    x5 == 250 - 3*x1 - 4*x2 - 5*x3,
    x6 == 300 - 3*x1 - 5*x2 - 4*x3,
    z  == 10*x1 + 15*x2 + 12*x3
], (x1, x4), (x2, x1))

x4 == 50 - x1 - x2 - x3
x5 == 250 - 3*x1 - 4*x2 - 5*x3
x6 == 300 - 3*x1 - 5*x2 - 4*x3
z == 10*x1 + 15*x2 + 12*x3

x1 == 50 - x2 - x3 - x4
x5 == 100 - x2 - 2*x3 + 3*x4
x6 == 150 - 2*x2 - x3 + 3*x4
z == 500 + 5*x2 + 2*x3 - 10*x4

x2 == 50 - x1 - x3 - x4
x5 == 50 + x1 - x3 + 4*x4
x6 == 50 + 2*x1 + x3 + 5*x4
z == 750 - 5*x1 - 3*x3 - 15*x4



## Reševanje duala linearnega programa

Poiščimo še optimalno rešitev duala prejšnjega linearnega programa.
$$
\begin{aligned}
\min &\ & 50 y_4 + 250 y_5 + 300 y_6 \\[1ex]
\text{p.p.} && y_4 + 3 y_5 + 3 y_6 &\ge 10 \\
&& y_4 + 4 y_5 + 5 y_6 &\ge 15 \\
&& y_4 + 5 y_5 + 4 y_6 &\ge 12 \\
&& y_4, y_5, y_6 &\ge 0
\end{aligned}
$$

In [16]:
spremenljivke(6, 0, predpona="y", ciljna="w z")
simpleksna_metoda([
    y1 == -10 + y0 + y4 + 3*y5 + 3*y6,
    y2 == -15 + y0 + y4 + 4*y5 + 5*y6,
    y3 == -12 + y0 + y4 + 5*y5 + 4*y6,
    w  ==     - y0,
    z  == -50*y4 - 250*y5 - 300*y6,
], (y0, y2), (y4, y0), DrugaFaza(y0))

w == -y0
y1 == -10 + y0 + y4 + 3*y5 + 3*y6
y2 == -15 + y0 + y4 + 4*y5 + 5*y6
y3 == -12 + y0 + y4 + 5*y5 + 4*y6
z == -50*y4 - 250*y5 - 300*y6

w == -15 - y2 + y4 + 4*y5 + 5*y6
y0 == 15 + y2 - y4 - 4*y5 - 5*y6
y1 == 5 + y2 - y5 - 2*y6
y3 == 3 + y2 + y5 - y6
z == -50*y4 - 250*y5 - 300*y6

w == -y0
y1 == 5 + y2 - y5 - 2*y6
y3 == 3 + y2 + y5 - y6
y4 == 15 - y0 + y2 - 4*y5 - 5*y6
z == -750 + 50*y0 - 50*y2 - 50*y5 - 50*y6

y1 == 5 + y2 - y5 - 2*y6
y3 == 3 + y2 + y5 - y6
y4 == 15 + y2 - 4*y5 - 5*y6
z == -750 - 50*y2 - 50*y5 - 50*y6

